In [1]:
%run ../../import_src.py

import lymedata
from lymedata import *
import constants
from constants import *

import tensorflow as tf
from tensorflow import keras
from keras import layers

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

2025-03-02 15:56:22.761565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-02 15:56:22.774047: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-02 15:56:22.777813: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 15:56:22.786661: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
tf.keras.backend.clear_session()

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

data = LymeData({CHRONIC, NEURO, MUSCULO},{ADDL_CIR},{NEURO, MUSCULO, BOTH, NEITHER}, defn=DEF_CNS1, drop_99=True)
print(data.df.shape)
print(data.df.columns)

X, Y = data.get_data_and_labels()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Scaling not helpful
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

for arr in [X_train, X_test, Y_train, Y_test]:
    print(arr.shape)
# Results of shape:
# (2878, 12)
# (720, 12)
# (2878, 4)
# (720, 4)
num_train_labels = X_train.shape[1]

# automatically one-hot-encoded
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(num_train_labels, )),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')
])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train,Y_train, epochs=100, batch_size=32) 

# 3 layers:
# 30 epochs: 63.1% train, 62.0 test
# 35 epochs: 0.6206 train, 0.6296 test
# 40 epochs: 0.6076 train, 0.6173

# 4 layers:
# 50 epochs: 63.3% train, 61.4% test

pred = model.predict(X_test)

Y_pred = np.argmax(pred, axis=1)
Y_true = np.argmax(Y_test, axis=1)

accuracy = accuracy_score(Y_true, Y_pred)
print(Y_pred)
print(f'Accuracy: {accuracy:.4f}')


/home/reu24lyme/alex/code_lib/src/lymedata.py:74: DtypeWarning: Columns (224,249,259,269,274,294,299,304,309,339,344,365,369,373,377,395,399,403,411,415,419,423,435) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Both Neuro and Mus: 2259
Only Neuro: 310
Only Mus: 762
Neither Neuro nor Mus: 632
Dropping 99
(1619, 12)
Index(['Bio Sex', 'Antibiotics', 'Times Infected', 'GROC', 'Bed Days',
       'Mental Health Days', 'Physical Health Days', 'Disability', 'neuro',
       'musculo', 'both', 'neither'],
      dtype='object')
(1295, 8)
(324, 8)
(1295, 4)
(324, 4)
Epoch 1/100


/home/reu24lyme/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4310 - loss: 1.2564
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6121 - loss: 1.0000
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5776 - loss: 1.0429
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6018 - loss: 0.9895
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6078 - loss: 0.9935
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6068 - loss: 0.9871
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6270 - loss: 0.9199
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6238 - loss: 0.9577
Epoch 9/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6152 - loss: 0.9519
Epoch 10/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6029 - loss: 0.9556
Epoch 11/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6094 - loss: 0.9703
Epoch 12/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5939 - l